In [1]:
import requests
from bs4 import BeautifulSoup as BS, element as BS_element
import lxml
import pandas as pd

In [2]:
# rutor_url = 'http://5.45.84.133/new/'
rutor_url = 'http://rutor.info/new'
host = '/'.join(rutor_url.split('/', maxsplit=3)[:3])
response = requests.get(rutor_url)
response.status_code

200

In [3]:
soup = BS(response.content, 'lxml')
main_table = soup.find('div', id='index')
dependent_tables: tuple[BS_element.Tag] = tuple(main_table.find('h2').find_next_siblings())

In [4]:
def _get_from_table(table: BS_element.Tag, group_name: str) -> pd.DataFrame:
    elements: tuple[BS_element.Tag] = tuple(table.find_all('td'))
    BS_data, BS_name, BS_size, BS_pirs = elements[0],elements[1],elements[-2],elements[-1]
    
    data = BS_data.get_text(strip=True)
    
    BS_name: BS_element.Tag = tuple(BS_name.find_all('a'))[-1]
    name = BS_name.get_text(strip=True)
    href = host + BS_name.get('href')

    size = BS_size.get_text(strip=True)
    
    up, down = tuple(map(int, BS_pirs.stripped_strings))
    
    return pd.DataFrame({
        'group': group_name,
        'data': data,
        'name': name,
        'size': size,
        'up': up,
        'down': down,
        'href': href
    }, index=[0])
    
    

def _get_table(elements: tuple[BS_element.Tag, BS_element.Tag]) -> list[pd.DataFrame]:
    bs_name, bs_table = elements
    group_name = bs_name.find('a').get_text(strip=True)
    group_tables: tuple[BS_element.Tag] = tuple(bs_table.find('tr', class_='backgr').find_next_siblings('tr'))
    
    ap_dataframe = []
    for item in group_tables:
        ap_dataframe.append(_get_from_table(item, group_name))
    
    return ap_dataframe
    
    
rearranged_tables: tuple[tuple[BS_element.Tag, BS_element.Tag]] = tuple(zip(dependent_tables[0::2], dependent_tables[1::2]))
ap_dataframe = []
for item in rearranged_tables:
    ap_dataframe.append(_get_table(item))

ap_dataframe = sum(ap_dataframe, [])
df = pd.concat(ap_dataframe, ignore_index=True, sort= False)

In [5]:
test = df['group'].unique()
print(test)
interesting = test[[0, 3, 6, 7, 10]]

['Зарубежные фильмы' 'Наши фильмы' 'Научно-популярные фильмы'
 'Зарубежные сериалы' 'Наши сериалы' 'Телевизор' 'Мультипликация' 'Аниме'
 'Иностранные релизы' 'Музыка' 'Игры' 'Софт' 'Спорт и Здоровье' 'Юмор'
 'Хозяйство и Быт' 'Книги' 'Другое']


In [6]:
my_df = df[df['group'].isin(interesting)]

cut = 40
my_df = my_df[(my_df['up'] >= cut) | (my_df['down'] >= cut)]
pd.set_option('display.max_colwidth', 1000)
display(my_df[my_df.columns[:-1]])

,group,data,name,size,up,down
11,Зарубежные фильмы,27 Май 23,"Подземелья и драконы: Честь среди воров / Dungeons & Dragons: Honor Among Thieves (2023) BDRip-AVC от DoMiNo & селезень | P | Jaskier, TVShows, LostFilm",3.14 GB,28,45
23,Зарубежные фильмы,27 Май 23,Остров иллюзий / Influencer (2022) WEB-DL 1080p | Sub,4.53 GB,103,52
25,Зарубежные фильмы,27 Май 23,Люби снова / Love Again (2023) WEB-DLRip от MegaPeer | Jaskier,1.46 GB,68,32
32,Зарубежные фильмы,27 Май 23,Профессор и призрачный убийца / The Ritual Killer (2023) BDRip-AVC от DoMiNo & селезень | D,1.46 GB,136,34
36,Зарубежные фильмы,27 Май 23,Дикая земля / Savageland (2015) WEB-DL 720p | L1,1.86 GB,47,5
37,Зарубежные фильмы,27 Май 23,Профессор и призрачный убийца / The Ritual Killer (2023) BDRip 1080p от New-Team | D,7.24 GB,117,53
39,Зарубежные фильмы,27 Май 23,Волшебная флейта / The Magic Flute (2022) WEB-DLRip от MegaPeer | D,1.46 GB,74,10
41,Зарубежные фильмы,27 Май 23,Профессор и призрачный убийца / The Ritual Killer (2023) BDRip от MegaPeer | D,1.46 GB,461,73
42,Зарубежные фильмы,27 Май 23,Профессор и призрачный убийца / The Ritual Killer (2023) BDRip от MegaPeer | D,745.91 MB,101,12
44,Зарубежные фильмы,27 Май 23,Волшебная флейта / The Magic Flute (2022) WEB-DL 1080p от селезень | D,5.03 GB,110,47


In [9]:
id = [284]
my_df.loc[id, 'href']

284    http://rutor.info/torrent/926404/solasta-crown-of-the-magister-lightbringers-edition-v-1.5.46-dlcs-2021-pc-repack-ot-fitgirl
Name: href, dtype: object